# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [115]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [116]:
stocks = pd.read_csv("sp_500_stocks.csv")
from config import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [117]:
symbol = "aapl"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [118]:
price = data["latestPrice"]
pe_ratio = data["peRatio"]

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [119]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [120]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    data[symbol]["quote"]["peRatio"],
                    "N/A",
                ],
                index=my_columns,
            ),
            ignore_index=True,
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,162.700,50.66,N/A
1,AAL,20.147,-4.1,N/A
2,AAP,226.750,24.48,N/A
3,AAPL,153.230,30.56,N/A
4,ABBV,118.470,31.02,N/A
...,...,...,...,...
496,YUM,131.390,30.43,N/A
497,ZBH,148.540,34.94,N/A
498,ZBRA,564.770,39.9,N/A
499,ZION,64.120,6.53,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [121]:
final_dataframe.sort_values("Price-to-Earnings Ratio", inplace=True)
final_dataframe = final_dataframe[final_dataframe["Price-to-Earnings Ratio"] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop(["index"], axis=1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,76.807,4.34,N/A
1,MPC,69.160,6,N/A
2,BIO,823.310,6.09,N/A
3,COF,154.326,6.14,N/A
4,PRU,116.960,6.17,N/A
5,ZION,64.120,6.53,N/A
6,CE,162.130,6.54,N/A
7,SYF,48.230,6.94,N/A
8,WHR,218.079,7.1,N/A
9,DFS,119.420,7.15,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [122]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [123]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [124]:
position_size = float(portfolio_size)/final_dataframe.shape[0]
for row in final_dataframe.index:
    final_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[row, "Price"])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,76.807,4.34,650
1,MPC,69.160,6,722
2,BIO,823.310,6.09,60
3,COF,154.326,6.14,323
4,PRU,116.960,6.17,427
5,ZION,64.120,6.53,779
6,CE,162.130,6.54,308
7,SYF,48.230,6.94,1036
8,WHR,218.079,7.1,229
9,DFS,119.420,7.15,418


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [125]:
symbol = "AAPL"
batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio
pe_ratio = data[symbol]["quote"]["peRatio"]

# Price-to-book ratio
pb_ratio = data["AAPL"]["advanced-stats"]["priceToBook"]

# Price-to-sales ratio
ps_ratio = data["AAPL"]["advanced-stats"]["priceToSales"]

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data["AAPL"]["advanced-stats"]["enterpriseValue"]
ebitda = data["AAPL"]["advanced-stats"]["EBITDA"]
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data["AAPL"]["advanced-stats"]["grossProfit"]
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [126]:
rv_colums = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "Price-to-Earnings Ratio",
    "PE Percentile",
    "Price-to-Book Ratio",
    "PB Percentile",
    "Price-to-Sales Ratio",
    "PS Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV Score",
]

rv_dataframe = pd.DataFrame(columns=rv_colums)
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        enterprise_value = data[symbol]["advanced-stats"]["enterpriseValue"]
        
        ebitda = data[symbol]["advanced-stats"]["EBITDA"]
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        gross_profit = data[symbol]["advanced-stats"]["grossProfit"]
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    "N/A",
                    data[symbol]["quote"]["peRatio"],
                    "N/A",
                    data[symbol]["advanced-stats"]["priceToBook"],
                    "N/A",
                    data[symbol]["advanced-stats"]["priceToSales"],
                    "N/A",
                    ev_to_ebitda,
                    "N/A",
                    ev_to_gross_profit,
                    "N/A",
                    "N/A",
                ],
                index=rv_colums,
            ),
            ignore_index=True,
        )

In [127]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,158.600,N/A,49.02,N/A,10.11,N/A,7.94,N/A,31.555302,N/A,15.319679,N/A,N/A
1,AAL,19.984,N/A,-4.12,N/A,-1.72,N/A,0.5146,N/A,-7.621734,N/A,1.527422,N/A,N/A
2,AAP,229.320,N/A,24.84,N/A,4.5,N/A,1.36,N/A,12.266989,N/A,2.828612,N/A,N/A
3,AAPL,154.041,N/A,30.44,N/A,38.78,N/A,7.42,N/A,22.438303,N/A,17.343264,N/A,N/A
4,ABBV,119.190,N/A,31.9,N/A,16.4,N/A,3.84,N/A,10.880770,N/A,7.854507,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,132.176,N/A,30.23,N/A,-4.75,N/A,5.92,N/A,20.932282,N/A,10.061033,N/A,N/A
497,ZBH,152.370,N/A,33.38,N/A,2.4,N/A,3.83,N/A,20.032204,N/A,6.578009,N/A,N/A
498,ZBRA,557.480,N/A,38.54,N/A,11.06,N/A,5.56,N/A,29.745068,N/A,12.523876,N/A,N/A
499,ZION,66.840,N/A,6.5,N/A,1.38,N/A,2.9,N/A,5.235643,N/A,2.739197,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [128]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,190.80,N/A,59.47,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,310.62,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,286.61,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.14,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
184,FLIR,58.51,N/A,32.67,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
188,FOX,38.02,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
202,GOOG,2987.40,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
323,MXIM,108.10,N/A,35.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
344,NWS,23.65,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
438,TROW,215.22,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [129]:
for col in rv_dataframe.columns:
    if rv_dataframe[col].isnull().any():
        rv_dataframe[col].fillna(rv_dataframe[col].mean(), inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,158.600,N/A,49.02,N/A,10.11,N/A,7.9400,N/A,31.555302,N/A,15.319679,N/A,N/A
1,AAL,19.984,N/A,-4.12,N/A,-1.72,N/A,0.5146,N/A,-7.621734,N/A,1.527422,N/A,N/A
2,AAP,229.320,N/A,24.84,N/A,4.50,N/A,1.3600,N/A,12.266989,N/A,2.828612,N/A,N/A
3,AAPL,154.041,N/A,30.44,N/A,38.78,N/A,7.4200,N/A,22.438303,N/A,17.343264,N/A,N/A
4,ABBV,119.190,N/A,31.90,N/A,16.40,N/A,3.8400,N/A,10.880770,N/A,7.854507,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,132.176,N/A,30.23,N/A,-4.75,N/A,5.9200,N/A,20.932282,N/A,10.061033,N/A,N/A
497,ZBH,152.370,N/A,33.38,N/A,2.40,N/A,3.8300,N/A,20.032204,N/A,6.578009,N/A,N/A
498,ZBRA,557.480,N/A,38.54,N/A,11.06,N/A,5.5600,N/A,29.745068,N/A,12.523876,N/A,N/A
499,ZION,66.840,N/A,6.50,N/A,1.38,N/A,2.9000,N/A,5.235643,N/A,2.739197,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [130]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [141]:
metrics = {
    "Price-to-Earnings Ratio": "PE Percentile",
    "Price-to-Book Ratio": "PB Percentile",
    "Price-to-Sales Ratio": "PS Percentile",
    "EV/EBITDA": "EV/EBITDA Percentile",
    "EV/GP": "EV/GP Percentile",
}

for metric, percentile in metrics.items():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, percentile] = stats.percentileofscore(
            rv_dataframe[metric], rv_dataframe.loc[row, metric]
        )/100

rv_dataframe

/home/m003/.pyenv/versions/3.9.1/envs/stocks_391/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.941,0,7.82,0.4,0.4898,0.16,0.3932,0.06,3.438695,0.16,0.397648,0.04,4.610778
1,AAL,19.984,0,-4.12,0.18,-1.7200,0.14,0.5146,0.18,-7.621734,0.06,1.527422,0.48,5.06986
2,PRU,113.870,0,6.28,0.22,0.6796,0.22,0.6634,0.28,4.669431,0.28,0.651552,0.06,5.309381
3,BA,216.090,0,-14.95,0.1,-8.7800,0.12,2.0100,0.9,-33.242756,0.02,-1358.686700,0.02,8.303393
4,KSS,52.320,0,11.36,0.66,1.4000,0.56,0.4052,0.1,4.129507,0.18,1.187260,0.28,8.323353
5,MCK,210.460,0,-7.67,0.14,-61.4100,0.04,0.1357,0.02,8.456538,0.88,3.005185,0.8,8.622754
6,ALL,126.760,0,10.17,0.56,1.4500,0.62,0.7858,0.38,2.344528,0.12,0.740747,0.1,8.622754
7,L,58.090,0,10.20,0.58,0.8359,0.32,0.9872,0.52,5.491956,0.36,0.999824,0.2,8.822355
8,RE,278.331,0,7.98,0.42,1.0048,0.36,0.9529,0.48,6.886752,0.64,0.920804,0.16,9.061876
9,LNC,74.500,0,10.33,0.62,0.6270,0.2,0.7017,0.3,7.740558,0.78,0.668079,0.08,9.221557


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [143]:
import statistics

value_percentiles = list(metrics.values())
for row in rv_dataframe.index:
    rv_dataframe.loc[row, "RV Score"] = statistics.mean(
        rv_dataframe.loc[row, value_percentiles]
    )
    
rv_dataframe

/home/m003/.pyenv/versions/3.9.1/envs/stocks_391/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.941,0,7.82,0.4,0.4898,0.16,0.3932,0.06,3.438695,0.16,0.397648,0.04,0.164
1,AAL,19.984,0,-4.12,0.18,-1.7200,0.14,0.5146,0.18,-7.621734,0.06,1.527422,0.48,0.208
2,PRU,113.870,0,6.28,0.22,0.6796,0.22,0.6634,0.28,4.669431,0.28,0.651552,0.06,0.212
3,BA,216.090,0,-14.95,0.1,-8.7800,0.12,2.0100,0.9,-33.242756,0.02,-1358.686700,0.02,0.232
4,KSS,52.320,0,11.36,0.66,1.4000,0.56,0.4052,0.1,4.129507,0.18,1.187260,0.28,0.356
5,MCK,210.460,0,-7.67,0.14,-61.4100,0.04,0.1357,0.02,8.456538,0.88,3.005185,0.8,0.376
6,ALL,126.760,0,10.17,0.56,1.4500,0.62,0.7858,0.38,2.344528,0.12,0.740747,0.1,0.356
7,L,58.090,0,10.20,0.58,0.8359,0.32,0.9872,0.52,5.491956,0.36,0.999824,0.2,0.396
8,RE,278.331,0,7.98,0.42,1.0048,0.36,0.9529,0.48,6.886752,0.64,0.920804,0.16,0.412
9,LNC,74.500,0,10.33,0.62,0.6270,0.2,0.7017,0.3,7.740558,0.78,0.668079,0.08,0.396


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [144]:
rv_dataframe.sort_values("RV Score", ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

/home/m003/.pyenv/versions/3.9.1/envs/stocks_391/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.941,0,7.82,0.4,0.4898,0.16,0.3932,0.06,3.438695,0.16,0.397648,0.04,0.164
1,AAL,19.984,0,-4.12,0.18,-1.7200,0.14,0.5146,0.18,-7.621734,0.06,1.527422,0.48,0.208
2,PRU,113.870,0,6.28,0.22,0.6796,0.22,0.6634,0.28,4.669431,0.28,0.651552,0.06,0.212
3,BA,216.090,0,-14.95,0.1,-8.7800,0.12,2.0100,0.9,-33.242756,0.02,-1358.686700,0.02,0.232
4,ALL,126.760,0,10.17,0.56,1.4500,0.62,0.7858,0.38,2.344528,0.12,0.740747,0.1,0.356
5,KSS,52.320,0,11.36,0.66,1.4000,0.56,0.4052,0.1,4.129507,0.18,1.187260,0.28,0.356
6,MCK,210.460,0,-7.67,0.14,-61.4100,0.04,0.1357,0.02,8.456538,0.88,3.005185,0.8,0.376
7,L,58.090,0,10.20,0.58,0.8359,0.32,0.9872,0.52,5.491956,0.36,0.999824,0.2,0.396
8,LNC,74.500,0,10.33,0.62,0.6270,0.2,0.7017,0.3,7.740558,0.78,0.668079,0.08,0.396
9,UAL,49.210,0,-4.97,0.16,2.8300,0.98,0.7703,0.36,-7.690599,0.04,1.527385,0.46,0.4


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [145]:
portfolio_input()

In [146]:
portfolio_size = float(portfolio_size)/rv_dataframe.shape[0]
for row in rv_dataframe.index:
    rv_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(portfolio_size/rv_dataframe.loc[row, "Price"])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.941,1855,7.82,0.4,0.4898,0.16,0.3932,0.06,3.438695,0.16,0.397648,0.04,0.164
1,AAL,19.984,2502,-4.12,0.18,-1.7200,0.14,0.5146,0.18,-7.621734,0.06,1.527422,0.48,0.208
2,PRU,113.870,439,6.28,0.22,0.6796,0.22,0.6634,0.28,4.669431,0.28,0.651552,0.06,0.212
3,BA,216.090,231,-14.95,0.1,-8.7800,0.12,2.0100,0.9,-33.242756,0.02,-1358.686700,0.02,0.232
4,ALL,126.760,394,10.17,0.56,1.4500,0.62,0.7858,0.38,2.344528,0.12,0.740747,0.1,0.356
5,KSS,52.320,955,11.36,0.66,1.4000,0.56,0.4052,0.1,4.129507,0.18,1.187260,0.28,0.356
6,MCK,210.460,237,-7.67,0.14,-61.4100,0.04,0.1357,0.02,8.456538,0.88,3.005185,0.8,0.376
7,L,58.090,860,10.20,0.58,0.8359,0.32,0.9872,0.52,5.491956,0.36,0.999824,0.2,0.396
8,LNC,74.500,671,10.33,0.62,0.6270,0.2,0.7017,0.3,7.740558,0.78,0.668079,0.08,0.396
9,UAL,49.210,1016,-4.97,0.16,2.8300,0.98,0.7703,0.36,-7.690599,0.04,1.527385,0.46,0.4


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [151]:
writer = pd.ExcelWriter("value_strategy.xlsx", engine="xlsxwriter")
rv_dataframe.to_excel(writer, sheet_name="Value Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [152]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [153]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for col in column_formats:
    writer.sheets["Value Strategy"].set_column(
        f"{col}:{col}",
        25,
        column_formats[col][1]
    )
    writer.sheets['Value Strategy'].write(
        f'{col}1',
        column_formats[col][0],
        column_formats[col][1]
    )

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [154]:
writer.save()